<a href="https://colab.research.google.com/github/SourabhMagadum/Hindi-Sentiment-Analysis/blob/master/Hindi_sentiment_analysis%20best.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# from tensorflow.python.keras import Sequential
from keras.layers import  Dense, Embedding, LSTM, Bidirectional,GRU
# from tensorflow.python.keras.layers import Dense, Embedding, LSTM, GRU 
#
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid
#
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from keras import optimizers
from keras import layers
# from tensorflow.python.keras import layers
import tensorflow as tf

In [15]:
import pandas as pd

# Assuming your CSV file is named 'HINDI_dataset.csv' in the same directory
df = pd.read_csv("/kaggle/input/actsa-lstm/data_with_sentiment_labels.csv", names=["review", "sentiment"],encoding='UTF-8')
df = df.dropna()
len(df['review'])
# Display the first few rows of the DataFrame
df = df.iloc[1:] 

In [16]:
df.head()

,review,sentiment
1,పరువునష్టాన్ని కలిగించే చర్యలను 1860 నాటి భారత...,neutral
2,వాహనంలో ఉన్న పోలీసు సిబ్బందికి బుల్లెట్ ప్రూఫ్...,positive
3,"దేశంపట్ల కానీ, ప్రజలపట్ల కానీ బాధ్యత కనిపించదు.",negative
4,ఒక పనిమనిషి కుమారుడు ఈ దేశ ప్రధాని కావడం అంబేద...,neutral
5,మత ప్రాతిపదికన మనుషుల్ని రెచ్చగొట్టే ఉగ్రవాదం ...,neutral


In [17]:
print(df["sentiment"])
df["sentiment"] = [0 if i== "negative" else 1 if i=="positive" else 2 for i in df["sentiment"]]

1        neutral
2       positive
3       negative
4        neutral
5        neutral
          ...   
5406    positive
5407     neutral
5408    positive
5409     neutral
5410     neutral
Name: sentiment, Length: 5410, dtype: object


In [18]:
df.head()

,review,sentiment
1,పరువునష్టాన్ని కలిగించే చర్యలను 1860 నాటి భారత...,2
2,వాహనంలో ఉన్న పోలీసు సిబ్బందికి బుల్లెట్ ప్రూఫ్...,1
3,"దేశంపట్ల కానీ, ప్రజలపట్ల కానీ బాధ్యత కనిపించదు.",0
4,ఒక పనిమనిషి కుమారుడు ఈ దేశ ప్రధాని కావడం అంబేద...,2
5,మత ప్రాతిపదికన మనుషుల్ని రెచ్చగొట్టే ఉగ్రవాదం ...,2


In [19]:
df.shape

(5410, 2)

In [20]:
y = df["sentiment"]
X = df.drop(["sentiment"], axis = 1)

In [21]:
# 70/15/15 train/test/val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

X_train.head()

,review
2007,ఐదేళ్లలో మరమ్మతులు పూర్తిచేస్తామని పేర్కొన్నారు.
4461,"కానీ, బ్యాంకులు కూడా తమను కాపాడుకోవడానికి అంతే..."
2688,ఏటా నమోదవుతున్న యాభై అరవైవేల కేసులు దోమల నివార...
1194,ఈ ఏడాది వర్షాలు సకాలంలో దండిగా కురవాలి.
3037,చాలా వర్గాల్లో నిర్లిప్తత చోటు చేసుకున్నది.


In [23]:
tk = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{"}~\t\n')


tk.fit_on_texts(X['review'])

# + 1 for unknown token
vocab_size = len(tk.word_index) +1

X_train_seq = tk.texts_to_sequences(X_train['review'])
X_test_seq = tk.texts_to_sequences(X_test['review'])
X_val_seq = tk.texts_to_sequences(X_val['review'])
# Initializing max length of sentence to 20 words
max_length = 20

In [24]:
vocab_size

24706

In [26]:
# tk.word_index

In [27]:
print(vocab_size)


24706


In [28]:
# X_train_seq

In [29]:
#padding the sequences to make all the input sequences of the same length

X_train_seq_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_seq_pad = pad_sequences(X_test_seq, maxlen=max_length,padding='post')
X_val_seq_pad = pad_sequences(X_val_seq, maxlen=max_length,padding='post')


In [30]:
from tensorflow.keras.utils import to_categorical


In [31]:

le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)
y_val_le = le.transform(y_val)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)
y_val_oh = to_categorical(y_val_le)

In [32]:
y_train_le

array([2, 0, 0, ..., 2, 0, 2])

In [33]:
X_train_seq_pad

array([[ 2764,  1981, 14129, ...,     0,     0,     0],
       [    9,  1289,     2, ...,     0,     0,     0],
       [  892, 16249,  4305, ...,     0,     0,     0],
       ...,
       [ 9778,   480,  2866, ...,  9782,  1484,    25],
       [ 1586,  2393,  8069, ...,     0,     0,     0],
       [  216,  5222,   815, ...,     0,     0,     0]], dtype=int32)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping

In [38]:
emb_dims = 256
model = Sequential()
model.add(Embedding(vocab_size, emb_dims,  embeddings_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(LSTM(units=16, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(3, activation='softmax'))
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
# Assuming X_train_seq_pad and y_train_oh are your training data (sequences and one-hot encoded labels)
history = model.fit(X_train_seq_pad, y_train_oh, epochs=32, batch_size=128,validation_data=(X_val_seq_pad, y_val_oh), shuffle=False)


Epoch 1/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 71ms/step - accuracy: 0.4103 - loss: 1.4634 - val_accuracy: 0.4783 - val_loss: 1.1442
Epoch 2/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.4548 - loss: 1.1235 - val_accuracy: 0.4812 - val_loss: 1.0696
Epoch 3/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.4762 - loss: 1.0593 - val_accuracy: 0.4957 - val_loss: 1.0458
Epoch 4/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.6021 - loss: 0.9220 - val_accuracy: 0.5174 - val_loss: 1.0755
Epoch 5/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.8160 - loss: 0.5894 - val_accuracy: 0.4986 - val_loss: 1.3222
Epoch 6/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9291 - loss: 0.3593 - val_accuracy: 0.4986 - val_loss: 1.5825
Epoch 7/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 63ms/step - accuracy: 0.9494 - loss: 0.2757 - val_accuracy: 0.5217 - val_loss: 1.6412
Epoch 8/32
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.9611 - loss: 0.2312 - val_accuracy: 0.5072 - v

In [41]:
from sklearn.metrics import f1_score, precision_score, recall_score
y_pred = model.predict(X_test_seq_pad)
y_pred_classes = np.argmax(y_pred, axis=1)  # Assuming y_pred is one-hot encoded
y_test_classes = le.inverse_transform(y_test_le)  # Convert back to original classes


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [42]:
f1 = f1_score(y_test_classes, y_pred_classes, average='macro')  # Macro averaging for imbalanced data (optional)
precision = precision_score(y_test_classes, y_pred_classes, average='macro')  # Macro averaging
recall = recall_score(y_test_classes, y_pred_classes, average='macro')  # Macro averaging

print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


F1 Score: 0.5071
Precision: 0.5156
Recall: 0.5039


In [ ]:
model.summary()


In [ ]:
results = model.evaluate(X_test_seq_pad, y_test_oh)
print('/n')
print('Test accuracy of word embeddings model: {0:.2f}%'.format(results[1]*100))

In [ ]:
results = model.evaluate(X_val_seq_pad, y_val_oh)
print('/n')
print('Validation accuracy of word embeddings model: {0:.2f}%'.format(results[1]*100))